### "Theory" recap

So guys, here's the thing, to summarize it the most.
The RBM is a particular case of Boltzmann Machine. It is basically a very special neural network, with just <b>two</b> layers. It has an input layer, called the "visible layer", and a hidden layer. <i>The important thing is that, in each layer, the neurons are NOT connected together on the same layer, but each of them is connected to ALL the others on the other layer</i>.

For instance, each neuron of the visible layer is connected to every neuron of the hidden layer. Conversely, each neuron of the hidden layer is connected to every neuron of the visible one. Because of this, each node (neuron) in one of the two layers is independent of the others in the same layers.

Also, there's no output layer, since what the RBM does is not discriminate within several possibilities but actually reconstruct a probability distribution (which is the input's). The reconstructed input is then compared with the real input and the difference (error) is computed. At each successive iteration, the error gets minimized thus the RBM (actually we maximize the log-likelihood which results in this effect) will be better and better at reconstructing the input. This is what it does in a very small nutshell.

Further, in our case the RBM is a <b>binary one</b>, meaning that each node (both visible and hidden) can only have two values: 0 or 1, or, optionally, -1 and 1 (the SPINS global variable that Baiesi put in his script).

To dwelve in a little more: the model (so the parameters that the net is searching for) are essentially $3$. These are: a weight matrix $W$ of L rows and M columns where L is the size of the inputs and M is the number of hidden layer neurons, a hidden layer bias vector $\vec{a}$ and a visible layer bias vector $\vec{b}$. These are collectivelly addressed as $\vec{\theta}$ in much of the established literature.

Initially, the weights are randomly chosen from a gaussian distribution and usually the biases are sampled in the same way (but sometimes as Baiesi did, they are fixed at certain arbitrary initial values, like all zeros or all ones). The whole thing happens in several iterations (epochs) and in two fundamental phases:

1) a <b>forward</b> pass: in this case the net computes the hidden activations starting from the input layer. Each input node is connected to all the hidden nodes and each of these connections has a weight $w_{ij}$ that is an element of the $W$ matrix. By convention $w_{ij}$ is the weight of the connection (i.e. the strength of the connection) between the i-th visible node and the j-th hidden node. So, the input value (0 or 1) of node i is multiplied by $w_{ij}$, added the bias $a$ and passed as argument to the activation function $S(x)$ which is a sigmoid function. This value is called $h_{ij}$ and is the activation of the hidden neuron. The sum of all the $h_{ij}$ on all $i$ gets us $h_j$ which is the activation of the j-th hidden neuron.

2) a <b>back</b> pass: in this case, the roles are exchanged. The hidden layer acts as input, using the same weight matrix W and it reconstructs the visible layer by multiplying each hidden node value by the weight $w_{ij}$ corresponding to the visible node being considered and then the bias $b$ is added and, again, everything is passed to the activation function $S(x)$, obtaining thus a reconstructed value of each input node. This is what Baiesi called "fantasy particles". Sometimes in the literature they are called $r$ nodes.

This process is called <b>Contrastive Divergence</b> (or CD) and can be repeated several times (in our case up to 5 times in the exercise). Now, after a given number of input samples are presented to the RBM (the so called minibatch), the optimization algorithm kicks in (in our case a simple SGD) and updates the values of the parameters of the model, shifting them in a way so that the log-likelihood is maximized throughout the iterations.

### Log Likelihood
To measure the performances of the model, one can compute the errors (quite simply the number of correctly/wrongly reconstructed input vectors). To compute the log-likelihood, I found these useful formulas that apply in our case and are like an "operative", more practical versions of what Baiesi used in the lectures:

$$
\ln{\mathcal{L}}(\mathbb{\theta}|\mathbb{v}) = \ln{\left[\frac{1}{Z}\cdot \sum_{\mathbb{h}} \exp{\left(-E(\mathbb{h},\mathbb{v})\right)}\right]}
$$

where $Z$ is the partition function, given by:

$$\boxed{
Z = \sum_{\mathbb{h},\mathbb{v}} \exp{\left(-E(\mathbb{h},\mathbb{v})\right)}
}
$$

Note that this function applies to a single input vector. In order to get the total likelihood, there's another catch.

In the case of $Z$, the sum is a double sum and it is to be understood as a double sum over all possible combinations of $h$ and $v$, meaning that if we have L visible nodes and M hidden nodes, we will have $2^{L+M}$ combinations that have to go inside here. So, if we had like L=4 and M=2, we will have, 16 possible vectors for the input and 4 vectors for the hidden states. Hence, given an input $\mathbb{v}$ we have to sum these contributions for that vector:

$$
\exp{\left(-E(h_0, \mathbb{v})\right)} + \exp{\left(-E(h_1, \mathbb{v})\right)} + \exp{\left(-E(h_2, \mathbb{v})\right)} + \exp{\left(-E(h_3, \mathbb{v})\right)}
$$

where $h_0 = (0,0)$, $h_1 = (0,1)$, $h_2 = (1,0)$ and $h_3 = (1,1)$.


The above formula is however written in this way:

$$
\ln{\mathcal{L}}(\mathbb{\theta}|\mathbb{v}) = \ln{\left[\sum_{\mathbb{h}}\exp{\left(-E(\mathbb{h},\mathbb{v})\right)}\right]} - \ln{Z}
$$

Now, to obtain the full likelihood (since we have more than 1 input vector) we can leverage the fact that the nodes are all independent within a layer. So the full likelihood would be the product of all the individual likelihoods. In this case we are considering the log-likelihood and hence we'll have the sum:

$$
\ln{\mathcal{L}(\mathbb{\theta})} = \sum_{\mathbb{v}}\left[ \ln{\left[\sum_{\mathbb{h}}\exp{\left(-E(\mathbb{h},\mathbb{v})\right)}\right]} - \ln{Z} \right]
$$

which finally is:

$$\boxed{
\ln{\mathcal{L}(\mathbb{\theta})} = \sum_{\mathbb{v}}\left[ \ln{\left[\sum_{\mathbb{h}}\exp{\left(-E(\mathbb{h},\mathbb{v})\right)}\right]}\right] - N\cdot\ln{Z}}
$$

where $N$ is the total number of input vectors $\mathbb{v}$. A reference for this formula is inside the pdfs that I put in the same folder of this notebook.

### The exercise
Regarding the exercise:
1. Point 1 requires us to add a little loop inside Baiesi's train procedure so that the whole forward and back passes cycle is repeated N times, with N being 1 initially and him wanting us to set it to 5. This is quite straightforward.

2. Point 2 requires us to apply the above formula to calculate the log-likelihood. This has to be computed in two scenarios: once for each epoch (using thus the values of the parameters at that epoch) and once for each minibatch in order to compare them (using the values at a particular minibatch, that is basically when the training procedure has read X values, where X is the size of the minibatch that Baiesi set at 500). In the first case the "time" index would be the epoch index and in the second the minibatch index.

3. Point 3 requires us to change the topology of the net, trying it out with various hidden layer sizes

4. Point 4 again we have to plot the log-likelihood in the case of M=3 with CD set to 1 and 5. Then for CD=1, plot the log-likelihood for M=3 and compare with the other Ms.

5. Point 5 requires us to compute the structure of the data-